In [345]:
import pandas as pd
import numpy as np
from matplotlib.patches import Rectangle
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patches as patches
import math
import sys


In [ ]:
######   task1

In [ ]:
# class of rectangle
class Rect:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
# calculate the perimeter of rectangle
    def perimeter(self):
        return 2 * (abs(self.x2 - self.x1) + abs(self.y2 - self.y1))
# Determine whether two rectangles overlap
    def is_overlap(self, rect):
      if self.y1 > rect.y2 or self.y2 < rect.y1 or self.x1 > rect.x2 or self.x2 < rect.y1:
        return False
      return True
# Determine whether another rectangle is contained by our rectangle
    def contain_rect(self, rect):
        return self.x1 < rect.x1 and self.y1 < rect.y1 and self.x2 > rect.x2 and self.y2 > rect.y2
# Determine whether our rectangle contains a point
    def has_point(self, point):
        return self.x1 <= point.x <= self.x2 and self.y1 <= point.y <= self.y2

    def __str__(self):
        return "Rect: ({}, {}), ({}, {})".format(self.x1, self.y1, self.x2, self.y2)


In [ ]:
# class of point   
class Point:
    def __init__(self, pointInfo):
        self.id = pointInfo[0]
        self.x = pointInfo[1]
        self.y = pointInfo[2]

    def __str__(self):
        return "Point #{}: ({}, {})".format(self.id, self.x, self.y)


# def sequential_query(points, query):
#     result = 0
#     for point in points:
#         if query.x1 <= point.x <= query.x2 and query.y1 <= point.y <= query.y2:
#             result = result + 1
#     return result


In [53]:
# class of node
class Node(object):
    def __init__(self, d, n, height):
        self.d = d
        self.n = n
        self.id = 0
        # for internal nodes
        self.child_nodes = []
        # for leaf nodespyth
        self.data_points = []
        self.parent_node = None
        self.MBR = Rect(-1, -1, -1, -1)
        self.height = height

    def add_point(self, point):
        # update in the right position to keep the list ordered
        self.add_points([point])
        pass

    def add_points(self, points):
        self.data_points += points
        # update MBR
        self.update_MBR()
        pass
    
    # Calculate the increment of perimeter after adding the point
    def perimeter_increase_with_point(self, point):
        x1 = point.x if point.x < self.MBR.x1 else self.MBR.x1
        y1 = point.y if point.y < self.MBR.y1 else self.MBR.y1
        x2 = point.x if point.x > self.MBR.x2 else self.MBR.x2
        y2 = point.y if point.y > self.MBR.y2 else self.MBR.y2
        return Rect(x1, y1, x2, y2).perimeter() - self.perimeter()

    def perimeter(self):
        # only calculate the half perimeter here
        return self.MBR.perimeter()

    #def is_underflow(self):
      # Determine whether the underflow, if it is a leaf node and the space is enough, then underflow
      # Or if it is a non leaf node and has enough space, then underflow
        #return (self.is_leaf() and len(self.data_points) < math.ceil(self.n / 2)) or \
               #(not self.is_leaf() and len(self.child_nodes) < math.ceil(self.d / 2))

    def is_overflow(self):
      # Determine whether the oerflow, if it is a leaf node and the space is enough, then oerflow
      # Or if it is a non leaf node and has enough space, then overflow
        return (self.is_leaf() and len(self.data_points) > self.n) or \
               (not self.is_leaf() and len(self.child_nodes) > self.d)

    def is_root(self):
      # Determine whether it is the root nod
        return self.parent_node is None

    def is_leaf(self):
        return len(self.child_nodes) == 0
    # add child node
    def add_child_node(self, node):
        self.add_child_nodes([node])
        pass

    def add_child_nodes(self, nodes):
        for node in nodes:
            node.parent_node = self
            self.child_nodes.append(node)
        self.update_MBR()
        pass
    # update MBR in order to ensure that MBR can contain all the points
    def update_MBR(self):
        if self.is_leaf():
            self.MBR.x1 = min([point.x for point in self.data_points])
            self.MBR.x2 = max([point.x for point in self.data_points])
            self.MBR.y1 = min([point.y for point in self.data_points])
            self.MBR.y2 = max([point.y for point in self.data_points])
        else:
            self.MBR.x1 = min([child.MBR.x1 for child in self.child_nodes])
            self.MBR.x2 = max([child.MBR.x2 for child in self.child_nodes])
            self.MBR.y1 = min([child.MBR.y1 for child in self.child_nodes])
            self.MBR.y2 = max([child.MBR.y2 for child in self.child_nodes])
        if self.parent_node and not self.parent_node.MBR.contain_rect(self.MBR):
            self.parent_node.update_MBR()
        pass

    # Get perimeter of an MBR formed by a list of data points
    @staticmethod
    def get_points_MBR_perimeter(points):
        x1 = min([point.x for point in points])
        x2 = max([point.x for point in points])
        y1 = min([point.y for point in points])
        y2 = max([point.y for point in points])
        return Rect(x1, y1, x2, y2).perimeter()

    @staticmethod
    def get_nodes_MBR_perimeter(nodes):
        x1 = min([node.MBR.x1 for node in nodes])
        x2 = max([node.MBR.x2 for node in nodes])
        y1 = min([node.MBR.y1 for node in nodes])
        y2 = max([node.MBR.y2 for node in nodes])
        return Rect(x1, y1, x2, y2).perimeter()
  


In [ ]:
class RegionTree:
# rtree class
    def __init__(self, d=4, n=4):
        self.d = d
        self.n = n
        self.root = Node(self.d, self.n, height = 1)

    def insert_point(self, point, cur_node=None):
        # init U as node
        #print("{} is leaf: {}".format(self.root, self.root.is_leaf()))
        if cur_node is None:
            cur_node = self.root
        #print("{} is leaf: {}".format(cur_node, cur_node.is_leaf()))
        # Insertion logic start
        if cur_node.is_leaf():
            cur_node.add_point(point)
            # handle overflow
            if cur_node.is_overflow():
                self.handle_overflow(cur_node)
        else:
            chosen_child = self.choose_best_child(cur_node, point)
            self.insert_point(point, cur_node=chosen_child)

    # Find a suitable one to expand:
    @staticmethod
    def choose_best_child(node, point):
        best_child = None
        best_perimeter = 0
        # Scan the child nodes
        
        for item in node.child_nodes:
            if node.child_nodes.index(item) == 0 or best_perimeter > item.perimeter_increase_with_point(point):
                best_child = item
                best_perimeter = item.perimeter_increase_with_point(point)
        return best_child

    def handle_overflow(self, node):
    
        node, new_node = self.split_leaf_node(node) if node.is_leaf() else self.split_internal_node(node)

        if self.root is node:
            self.root = Node(self.d, self.n, height=node.height+1)
            self.root.add_child_nodes([node, new_node])
        else:
            node.parent_node.add_child_node(new_node)
            if node.parent_node.is_overflow():
                self.handle_overflow(node.parent_node)

    # split leaf node
    def split_leaf_node(self, node):
        m = len(node.data_points)
        best_perimeter = -1
        best_set_1 = []
        best_set_2 = []
        # Run x axis, and  sort all poins by x
        all_point_sorted_by_x = sorted(node.data_points, key=lambda point: point.x)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_point_1 = all_point_sorted_by_x[:i]
            list_point_2 = all_point_sorted_by_x[i:]
            temp_sum_perimeter = Node.get_points_MBR_perimeter(list_point_1) \
                                 + Node.get_points_MBR_perimeter(list_point_2)
            if best_perimeter == -1 or best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_point_1
                best_set_2 = list_point_2
        # Run y axis, and  sort all poins by y
        all_point_sorted_by_y = sorted(node.data_points, key=lambda point: point.y)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_point_1 = all_point_sorted_by_y[:i]
            list_point_2 = all_point_sorted_by_y[i:]
            temp_sum_perimeter = Node.get_points_MBR_perimeter(list_point_1) \
                                 + Node.get_points_MBR_perimeter(list_point_2)
            if best_perimeter == -1 or best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_point_1
                best_set_2 = list_point_2
        node.data_points = best_set_1
        node.update_MBR()
        new_node = Node(self.d, self.n, height = node.height)
        new_node.add_points(best_set_2)
        return node, new_node

    # split non leaf node
    def split_internal_node(self, node):
        m = len(node.child_nodes)
        best_perimeter = -1
        best_set_1 = []
        best_set_2 = []
        # Run x axis
        all_node_sorted_by_x = sorted(node.child_nodes, key=lambda child: child.MBR.x1)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_node_1 = all_node_sorted_by_x[:i]
            list_node_2 = all_node_sorted_by_x[i:]
            temp_sum_perimeter = Node.get_nodes_MBR_perimeter(list_node_1) \
                                 + Node.get_nodes_MBR_perimeter(list_node_2)
            if best_perimeter == -1 or best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_node_1
                best_set_2 = list_node_2
                # Run y axis
        all_node_sorted_by_y = sorted(node.child_nodes, key=lambda child: child.MBR.y1)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_node_1 = all_node_sorted_by_y[:i]
            list_node_2 = all_node_sorted_by_y[i:]
            temp_sum_perimeter = Node.get_nodes_MBR_perimeter(list_node_1) \
                                 + Node.get_nodes_MBR_perimeter(list_node_2)
            if best_perimeter == -1 or best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_node_1
                best_set_2 = list_node_2
        node.child_nodes = best_set_1
        node.update_MBR()
        new_node = Node(self.d, self.n, height = node.height)
        new_node.add_child_nodes(best_set_2)
        return node, new_node

    # Take in a Rectangle and return number of data point that is covered by the R tree.
    def region_query(self, rect, node=None):
        # initiate with root
        if node is None:
            node = self.root

        if node.is_leaf():
            # print("get here")
            count = 0
            for point in node.data_points:
                if rect.has_point(point):
                    count += 1
            return count
        else:
            print([child.MBR for child in node.child_nodes])
            total = 0
            for child in node.child_nodes:
                print("{} and {} is overlapped {}".format(rect, child.MBR, rect.is_overlap(child.MBR)))
                if rect.is_overlap(child.MBR):
                    total += self.region_query(rect, child)
            return total



In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/AllPOI Locatoion Only.csv', delimiter=',', names=["latitude", "longitude"])
data


,latitude,longitude
0,116.303382,39.908620
1,116.802076,40.335921
2,116.807570,40.362750
3,116.839759,40.371410
4,117.105753,40.140551
...,...,...
182318,117.240095,40.615747
182319,117.111129,40.213956
182320,116.796412,40.562527
182321,116.459869,39.852378


In [ ]:

points= np.array(data)
points = points.tolist()


In [ ]:
############  task2

In [376]:
pointsNew = []
index = 0
for point in points:
    point = [index] + point
    pointsNew.append(point)
    index += 1
#print(pointsNew)

tree = RegionTree(2, 10)
for point in pointsNew:
    tree.insert_point(Point(point))



In [ ]:
# the height of Rtree
print('the height of Rtree is:{}'.format(tree.root.height))

the height of Rtree is:7


In [ ]:
# find the number of non_leaf nodes and the number of leaf nodes
search_queue = [tree.root]
#print(search_queue)
searched = []
non_leaf = []
leaf = []
while search_queue:
    node = search_queue.pop(0)
    #print(node)
    
    if not node in searched:
        if node.is_leaf():
            leaf.append(node)
            #print(node)
        else:
            search_queue += node.child_nodes
            #print(node.child_nodes)
            non_leaf.append(node)
            #print(node)
        
        searched.append(node)

print('the number of non-leaf nodes is:{}'.format(len(non_leaf)))
print('the number of leaf nodes is:{}'.format(len(leaf)))

the number of non-leaf nodes is:926
the number of leaf nodes is:2740


In [ ]:
print(non_leaf)

[<__main__.Node object at 0x7f12faafd350>, <__main__.Node object at 0x7f12fb7ac110>, <__main__.Node object at 0x7f12faf07650>, <__main__.Node object at 0x7f12fbc78b50>, <__main__.Node object at 0x7f12fb32a810>, <__main__.Node object at 0x7f12faa9fc10>, <__main__.Node object at 0x7f12fa597cd0>, <__main__.Node object at 0x7f12fa3f7dd0>, <__main__.Node object at 0x7f12fb606410>, <__main__.Node object at 0x7f12fac18250>, <__main__.Node object at 0x7f12fb614110>, <__main__.Node object at 0x7f12fbab8090>, <__main__.Node object at 0x7f12fadd5090>, <__main__.Node object at 0x7f12fa9e9c50>, <__main__.Node object at 0x7f12fbd364d0>, <__main__.Node object at 0x7f12faa5bb90>, <__main__.Node object at 0x7f12fa597c90>, <__main__.Node object at 0x7f12fb096b10>, <__main__.Node object at 0x7f12fa20d710>, <__main__.Node object at 0x7f12fb34de90>, <__main__.Node object at 0x7f12fb614050>, <__main__.Node object at 0x7f12fb2d3550>, <__main__.Node object at 0x7f12f9ee8f90>, <__main__.Node object at 0x7f12fa

In [ ]:
# calculate the space utilisation of leaf nodes
spaceUtilisationList = []
for node in leaf:
    spaceUtilisation = len(node.data_points) / node.n
    spaceUtilisationList.append(spaceUtilisation)

capacity1 = 0
capacity2 = 0
capacity3 = 0
capacity4 = 0
for i in spaceUtilisationList:
    if i < 0.25:
        capacity1 += 1
    elif i < 0.5:
        capacity2 += 1
    elif i < 0.75:
        capacity3 += 1
    else:
        capacity4 += 1

print('0~25%: {}, 25~50%: {}, 50~75%: {}, 75~100%: {}' .format(capacity1, capacity2, capacity3, capacity4))

0~25%: 0, 25~50%: 399, 50~75%: 1509, 75~100%: 832


In [ ]:
# define a function to find whose child nodes is overlapped 
def MBRoverlap(node):
  child_nodes = node.child_nodes
  height = node.height
  for i in range(len(child_nodes)-1):
    node1= child_nodes[i]
    for j in range(i+1,len(child_nodes)):
      node2=child_nodes[j]
      if node1.MBR.is_overlap(node2.MBR):
        return node,height
  return [],height

In [ ]:

search_queue=[tree.root]
heightList={}

while search_queue:

    node = search_queue.pop(0)
    node_list = []
    overlapNode,height = MBRoverlap(node)
    heightList[height] = []
    node_list.append(overlapNode)
    #print('node_list:{}'.format(node_list))
    heightList[height] += node_list
    #print('heightList:{}'.format(heightList))
    search_queue += node.child_nodes

In [ ]:
# non-leaf nodes in the R-tree containing at least one pair of MBRs that overlap
heightList


{1: [[]],
 2: [<__main__.Node at 0x7f12fa9e9b50>],
 3: [<__main__.Node at 0x7f12fb6ede90>],
 4: [<__main__.Node at 0x7f12fad58f50>],
 5: [<__main__.Node at 0x7f12fa9e9c50>],
 6: [<__main__.Node at 0x7f12faf07650>],
 7: [<__main__.Node at 0x7f12faafd350>]}

In [ ]:
#########   task3 and task4

In [90]:
def distance(data_points, query_point):
    """
    Calculate the distance between 2 points

    Returns the square distance between the two.
    """
    answer = math.sqrt((data_points.x-query_point[0]) ** 2 + (data_points.y-query_point[1]) ** 2)
    
    return answer

In [91]:
def min_dist(query_point, Rect):
    mindist = 0
    # component on x axis
    if query_point[0] < Rect.x1:
        mindist += (Rect.x1 - query_point[0])**2
    elif query_point[0] > Rect.x2:
        mindist += (query_point[0] - Rect.x2)**2
    # component on y axis
    if query_point[1] < Rect.y1:
        mindist += (Rect.y1 - query_point[1])**2
    elif query_point[1] > Rect.y2:
        mindist += (query_point[1] - Rect.y2)**2
    # if the query point is inside of the node, return 0
    return math.sqrt(mindist)

In [92]:
def min_max_dist(query_point, Rect):

    S = 0
    px = query_point[0]
    py = query_point[1]
    rm1 = None
    rm2 = None
    rM1 = None
    rM2 = None
    # for i in range(2):
    s1, s2 = Rect.x1, Rect.y1
    t1, t2 = Rect.x2, Rect.y2


    if px <= (s1 + t1) / (2.0):
      rm1 = s1
    else:
      rm1 = t1
    
    if py <= (s2 + t2) / (2.0):
      rm2 = s2
    else:
      rm2 = t2
    

    if px >= (s1 + t1) / (2.0):
      rM1 = s1
    else:
      rM1 = t1
    
    if py >= (s2 + t2) / (2.0):
      rM2 = s2
    else:
      rM2 = t2
    
    max_dist_squared1 = (px - rm1) ** 2 + (py - rM2) ** 2
    max_dist_squared2 = (py - rm2) ** 2 + (px - rM1) ** 2
    min_max_dist_squared = min(max_dist_squared1, max_dist_squared2)
    min_max_dist = math.sqrt(min_max_dist_squared)
    return min_max_dist

In [93]:
# find the nearest neighbour of a given query point based on the R-tree
def NNandprune(rtreenode, query_point):
    global nearestdistance
    global results
    global visitednodes
    global calculatedpoint
    global prunedmbr
    global nodelist
    global index
    visitednodes += 1
    # if leaf node
    if rtreenode[0][1].is_leaf():
        node = rtreenode[0][1]# remove the first element 
        del rtreenode[0]
        for point in node.data_points:
            new_dis = distance(point, query_point)
            calculatedpoint += 1
            if new_dis <= nearestdistance:
                results.clear()
                results.append(point)
                if new_dis < nearestdistance:
                   nearestdistance = new_dis
        return 
   
    # for non-leaf nodes
    else:
        node = rtreenode[0][1]
        # store minDist and node in ABL and sort it by minDist
        ABL = [(min_dist(query_point,child.MBR), child) for child in node.child_nodes]
        ABL = sorted(ABL, key=lambda t: t[0])
        ABL_new = []
        del rtreenode[0]
        while (len(ABL) > 1):
            MBR0 = ABL[0];MBR1 = ABL[1]
            # use pruning rule 1
            if MBR0[0] > min_max_dist(query_point,MBR1[1].MBR):
               ABL.pop(0);prunedmbr += 1
            elif MBR1[0] > min_max_dist(query_point,MBR0[1].MBR):
               ABL.pop(1);prunedmbr += 1
            else:
                # use pruning rule 3
                if node not in nodelist.keys():
                   nodelist[node] = {}

                if MBR0 in nodelist[node].keys():p0 = nodelist[node][MBR0]
                else:
                   NNandprune([MBR0], query_point);p0 = results[0];nodelist[node][MBR0] = p0    
                   index += 1
                if MBR1 in nodelist[node].keys():p1 = nodelist[node][MBR1]
                else:
                   NNandprune([MBR1], query_point);p1 = results[0];nodelist[node][MBR1] = p1

                if MBR1[0] > distance(p0, query_point):
                   ABL.pop(1);prunedmbr += 1;calculatedpoint += 1
                elif MBR0[0] > distance(p1, query_point):
                   ABL.pop(0);prunedmbr += 1;calculatedpoint += 1
                elif distance(p1, query_point) > min_max_dist(query_point,MBR0[1].MBR):
                     ABL.pop(1);calculatedpoint += 1
                elif distance(p0, query_point) > min_max_dist(query_point,MBR1[1].MBR):
                     ABL.pop(0);calculatedpoint += 1
                else:
                     abl = ABL.pop(0);ABL_new.append(abl)
        ABL += ABL_new
        rtreenode += ABL
        

    if rtreenode == []:
        return
    if nearestdistance < rtreenode[0][0]:
        return
    NNandprune(rtreenode, query_point)
    return

In [377]:
query_point=[116,	43]
visitednodes=0
calculatedpoint=0
prunedmbr=0
nodelist={}
nearestdistance = float('inf')
results=[]
index=0
NNandprune([(0, tree.root)], query_point)


In [378]:
print('the point p in D that is the nearest neighbour of q:{}'.format(results[0]))

the point p in D that is the nearest neighbour of q:Point #121597: (116.63379499999999, 41.029933)


In [379]:
print('nearest distance:{}'.format(nearestdistance))

nearest distance:2.0695072086160975


In [380]:
print('the number of visited nodes:{}'.format(visitednodes))

the number of visited nodes:241


In [381]:
print(' the number of points where actual distance to q is calculated.:{}'.format(calculatedpoint))

 the number of points where actual distance to q is calculated.:46


In [382]:
print('the number of pruned mbr:{}'.format(prunedmbr))

the number of pruned mbr:178
